In [80]:
import glob
import os
import numpy as np
from skimage import io, transform, exposure, restoration, filters
from skimage.measure import regionprops
from scipy import ndimage
import matplotlib.pyplot as plt
import tqdm
import pims
import panel as pn

from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Spectral11
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10
from bokeh.models import Range1d
from bokeh.models import Legend

import pandas as pd


import bokeh.io
from bokeh.plotting import figure, show


from bokeh.io import output_notebook
output_notebook()

pn.extension()

Loading BokehJS ...

In [22]:
df = pd.read_csv('/Users/scliu/Documents/q-bio/correlation.csv', encoding = "ISO-8859-1")

In [23]:
df


,label,time,length
0,low,1,10.75
1,low,1,6.52
2,low,1,12.40
3,low,1,8.23
4,low,1,10.44
...,...,...,...
63,high,15,248.20
64,high,20,260.30
65,high,20,248.20
66,high,25,262.40


In [30]:
df_low = df.loc[df["label"] == "low"]
df_mid = df.loc[df["label"] == "mid"]
df_high = df.loc[df["label"] == "high"]

In [33]:
df_low.loc[df["time"] == 1, "length"]

0    10.75
1     6.52
2    12.40
3     8.23
4    10.44
Name: length, dtype: float64

In [37]:
df_low.time.unique()

array([ 1,  5, 10, 15, 20, 25])

In [92]:
p = figure(width=600, height=300, title="Feature distance")

colors = ["olive", "darkred", "goldenrod", "skyblue", "orange", "salmon"]


for i, time in enumerate(list(df_low.time.unique())):
    r1=p.cross(x = time, y = df_low.loc[df["time"] == time, "length"], color = colors[1], size = 10, legend_label="low intensity")
    r2=p.circle(x = time, y = df_mid.loc[df["time"] == time, "length"], color = colors[2], size = 10, legend_label="mid intensity")
    r3=p.triangle(x = time, y = df_high.loc[df["time"] == time, "length"], color = colors[3], size = 10, legend_label="high intensity")

p.legend.location = "top_left"
p.xaxis.axis_label = 'time (minutes)'
p.yaxis.axis_label = 'distance (µm)'



In [93]:
show(p)

In [66]:
base, lower, upper

(['low', 'mid', 'high'], [nan, nan, nan], [nan, nan, nan])

In [21]:
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show
from bokeh.sampledata.autompg import autompg as df

colors = ["red", "olive", "darkred", "goldenrod", "skyblue", "orange", "salmon"]

p = figure(width=600, height=300, title="Years vs mpg with Quartile Ranges")

base, lower, upper = [], [], []

for i, year in enumerate(list(df.yr.unique())):
    year_mpgs = df[df['yr'] == year]['mpg']
    mpgs_mean = year_mpgs.mean()
    mpgs_std = year_mpgs.std()
    lower.append(mpgs_mean - mpgs_std)
    upper.append(mpgs_mean + mpgs_std)
    base.append(year)

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

p.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)

for i, year in enumerate(list(df.yr.unique())):
    y = df[df['yr'] == year]['mpg']
    color = colors[i % len(colors)]
    p.circle(x=year, y=y, color=color)

df

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
388,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
389,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
390,28.0,4,120.0,79,2625,18.6,82,1,ford ranger
